<a href="https://colab.research.google.com/github/betegon/16-QAM-Detection-MIMO/blob/master/TAC_MIMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np


## EJEMPLO CON 2 antenas


In [0]:
k = 2 # 2 antennas

In [40]:
s_matrix = np.array([[-3, -1],[1, 3]])
print(s_matrix)
s = (np.array([s_matrix.flatten('F')])).transpose()
print("s\n",s)
t = s**2
print(t)

[[-3 -1]
 [ 1  3]]
s
 [[-3]
 [ 1]
 [-1]
 [ 3]]
[[9]
 [1]
 [1]
 [9]]


In [41]:
# CREACIÓN DE H 

k = 2  # number of antennas
# H is CN(0,1)
mu = np.zeros(2*k)
sigma = np.ones((k,k*2))


H = (1/np.sqrt(2))*np.random.normal(loc=mu, scale=sigma, size=(k,k*2)).view(np.complex128) 
print(H)
a = np.column_stack((np.real(H), -np.imag(H)))
print("Fila1: \n",a,"\n")
b = np.column_stack((np.imag(H), np.real(H)))
print("Fila2\n",b,"\n")
H_expanded = np.concatenate((a,b), axis=0)
print("Matriz H expandida\n",H_expanded)

[[-0.45220356-0.02024426j -0.38538491+1.37227257j]
 [-0.1527265 +0.09479141j  0.1442295 -0.24541657j]]
Fila1: 
 [[-0.45220356 -0.38538491  0.02024426 -1.37227257]
 [-0.1527265   0.1442295  -0.09479141  0.24541657]] 

Fila2
 [[-0.02024426  1.37227257 -0.45220356 -0.38538491]
 [ 0.09479141 -0.24541657 -0.1527265   0.1442295 ]] 

Matriz H expandida
 [[-0.45220356 -0.38538491  0.02024426 -1.37227257]
 [-0.1527265   0.1442295  -0.09479141  0.24541657]
 [-0.02024426  1.37227257 -0.45220356 -0.38538491]
 [ 0.09479141 -0.24541657 -0.1527265   0.1442295 ]]


In [0]:
def awgn(signal, snr, seed=False):
    """ Generate Aditive White Gaussian Noise (AWGN) from an input signal.

    Args:
        signal  (np.array): Input signal. e.g. signal = np.array([[1, 2.3, -1]])
        snr        (float): Signal-to-noise ratio in dB.
        seed        (bool): Set seed to reproduce results (good for debugging).
                            Defaults to False.

    Returns:
        noise (np.array): AWGN generated from an input signal.

    """
    if seed: np.random.seed(1)
    sigpower = sum([ math.pow(abs(signal[i]),2) for i in range(len(signal)) ])
    sigpower = sigpower / len(signal)
    noisepower = sigpower / (math.pow(10,snr/10))
    noise = np.random.normal(0, np.sqrt(noisepower), len(signal))
    return np.array([noise]).T

In [43]:
# y = H*s+w;      
mu = np.zeros(k)
sigma = np.ones((k,k))
snr = 20

noise = awgn(s,snr)
# noise = np.random.normal(loc=mu, scale=sigma, size=(k,k)).view(np.complex128) 
# noise = np.concatenate((np.real(noise), np.imag(noise)), axis=0)



y = H_expanded.dot(s) + noise
print("\nH\n",H_expanded)
print("\ns\n",s)
# print("\nnoise\n",noise)
print("\nnoise \n", noise)
print("\ny\n",y)


H
 [[-0.45220356 -0.38538491  0.02024426 -1.37227257]
 [-0.1527265   0.1442295  -0.09479141  0.24541657]
 [-0.02024426  1.37227257 -0.45220356 -0.38538491]
 [ 0.09479141 -0.24541657 -0.1527265   0.1442295 ]]

s
 [[-3]
 [ 1]
 [-1]
 [ 3]]

noise 
 [[-0.21406218]
 [ 0.12826537]
 [ 0.13233666]
 [-0.17464794]]

y
 [[-3.37989837]
 [ 1.56171548]
 [ 0.86139084]
 [-0.11902375]]


In [48]:
## CALCULO DE W (variable a minimizar)
w = np.concatenate((s,t,[[1]]))
w_transpose = w.transpose()
print("w\n",w)
print("w_transpose\n",w_transpose)

W = w.dot(w_transpose)
print("\nBig W\n",W)
print("\nBig W\n",W.shape)

w
 [[-3]
 [ 1]
 [-1]
 [ 3]
 [ 9]
 [ 1]
 [ 1]
 [ 9]
 [ 1]]
w_transpose
 [[-3  1 -1  3  9  1  1  9  1]]

Big W
 [[  9  -3   3  -9 -27  -3  -3 -27  -3]
 [ -3   1  -1   3   9   1   1   9   1]
 [  3  -1   1  -3  -9  -1  -1  -9  -1]
 [ -9   3  -3   9  27   3   3  27   3]
 [-27   9  -9  27  81   9   9  81   9]
 [ -3   1  -1   3   9   1   1   9   1]
 [ -3   1  -1   3   9   1   1   9   1]
 [-27   9  -9  27  81   9   9  81   9]
 [ -3   1  -1   3   9   1   1   9   1]]

Big W
 (9, 9)


In [45]:
y


array([[-3.37989837],
       [ 1.56171548],
       [ 0.86139084],
       [-0.11902375]])

In [54]:
# Cálculo de la matriz a multiplicar por H

A11 = H_expanded.T.dot(H_expanded) # Check if it is really doing the transpose of H_expanded
A12 = np.zeros((2*k,2*k))
A13 = -H_expanded.T.dot(y) # Check if it is really doing the transpose of H_expanded
A1X = np.column_stack((np.column_stack((A11, A12)), A13))
print("\nFILA 1")
print("\n",A1X)
print("\n",A11)
print("\n",A12)
print("\n",A13)

print("\nFILA 2")
A2X = np.zeros((2*k,A1X.shape[1]))
print("\n",A2X)

print("\nFILA 3")
A31 = (-y.T).dot(H_expanded)
print("\n",A31)
A32= np.zeros((1,2*k))
print("\n",A32)
A33 = (y.T).dot(y)
print("\n",A33)

A3X = np.column_stack((np.column_stack((A31, A32)), A33))
print("\n",A3X)

A = np.concatenate((A1X,A2X,A3X))
print("\n",A)
print("\n",A.shape)

# A = [H'*H zeros(2*N,2*N) -H'*y; zeros(2*N,4*N+1); -y'*H zeros(1,2*N) y'*y];



FILA 1

 [[ 2.37208687e-01  1.01200741e-01 -2.11825039e-19  6.04538483e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.26116611e+00]
 [ 1.01200741e-01  2.11268499e+00 -6.04538483e-01 -8.09511849e-19
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.73908069e+00]
 [-2.11825039e-19 -6.04538483e-01  2.37208687e-01  1.01200741e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   5.87806670e-01]
 [ 6.04538483e-01 -8.09511849e-19  1.01200741e-01  2.11268499e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -4.67227893e+00]]

 [[ 2.37208687e-01  1.01200741e-01 -2.11825039e-19  6.04538483e-01]
 [ 1.01200741e-01  2.11268499e+00 -6.04538483e-01 -8.09511849e-19]
 [-2.11825039e-19 -6.04538483e-01  2.37208687e-01  1.01200741e-01]
 [ 6.04538483e-01 -8.09511849e-19  1.01200741e-01  2.11268499e+00]]

 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

 [[-1.26116611]
 [-2.73908069]
 [ 0.58780667]
 [-4

array([[ 0.17392793,  0.97637835, -0.14898517,  0.74712049,  0.        ,
         0.        ,  0.        ,  0.        , -3.17580974],
       [-0.05797598, -0.32545945,  0.04966172, -0.24904016,  0.        ,
         0.        ,  0.        ,  0.        ,  1.05860325],
       [ 0.05797598,  0.32545945, -0.04966172,  0.24904016,  0.        ,
         0.        ,  0.        ,  0.        , -1.05860325],
       [-0.17392793, -0.97637835,  0.14898517, -0.74712049,  0.        ,
         0.        ,  0.        ,  0.        ,  3.17580974],
       [-0.52178379, -2.92913504,  0.44695552, -2.24136146,  0.        ,
         0.        ,  0.        ,  0.        ,  9.52742922],
       [-0.05797598, -0.32545945,  0.04966172, -0.24904016,  0.        ,
         0.        ,  0.        ,  0.        ,  1.05860325],
       [-0.05797598, -0.32545945,  0.04966172, -0.24904016,  0.        ,
         0.        ,  0.        ,  0.        ,  1.05860325],
       [-0.52178379, -2.92913504,  0.44695552, -2.24136146,  0

In [0]:
testing_api